In [ ]:
# pip install shapely  # Установить библиотеку shapely если она не установлена
from shapely.geometry import Polygon, Point #, MultiPolygon, GeometryCollection
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# ARC Аркадий Соловьев
# Скрипт нарезки файлов аннотаций для варианта нарезки кадров с нахлестом
# Аннотируется только целая капуста. Если полигон с капустой в пределах квадрата 640 х 640
# режется границами квадрата, то этот полигон не аннотируется.
# Применять для средних планов, где капусты много, но отличительные признаки плохой
# и хорошей капусты еще видны.
# Этот скрипт должен использоваться совместно со скриптом нарезки кадров с нахлестом
width = 1920      # Ширина исходного кадра
height = 1080     # Высота исходного кадра
square_size = 640 # Размер квадрата, на который нарезан кадр
y_step = 440      # Шаг по вертикали

# Это заглушка, пока она не работает
# Если использовать скрипт для нарезки кадров с растяжением кадра по вертикали, то
# необходимо указать шаг по вертикали y_step = 640 и размер по вертикали height = 1280
# снять комментарий у этих двух строк:
# y_step = 640
# height = 1280


# Указать путь к папке с исходными аннотациями
labels_folder = "/content/drive/MyDrive/1 Исходные файлы для Датасета 4.0/far/train/labels"
# Указать путь к папке для новых аннотаций
new_labels_folder = "/content/drive/MyDrive/test/new_labels_3"

# Функция пересчета относительных координат в пиксели
def convert_relative_to_pixels(coordinates):
    # Функция принимает: coordinates - относительные координаты вершин полигонов в кадре
    # (в долях длины и ширины кадра), width, height - ширина и высота кадра в пикселях

    pixel_coords = [] # Список координат вершин полигонов в пикселях
    for coord in coordinates:       # Проходим по относительным координатам
        x = int(coord[0] * width)   # Относительные координаты домножаем на размер кадра
        y = int(coord[1] * height)  #
        pixel_coords.append((x, y)) # Координаты в пикселях

    return pixel_coords # Функция возвращает координаты вершин полигогов в пикселях

# Функция пересчета абсолютных координат полигонов в пикселях в относительные координаты в долях текущего кавдрата
def convert_coordinates_to_ratios(sliced_polygons, n): #
    # Функция принимает: sliced_polygons - данные полигонов (список координат x и y  вершин полигонов),
    # n - текущий номер квадрата в кадре

    transformed_polygons = []       # Список координат вершин полигонов в долях квадрата

    for polygon in sliced_polygons: # Проходим по полигонам
        transformed_polygon = []
        polygon_cut = False         # Предположение, что полигон не пересекает квадрат
        for x, y in polygon:        # Проходим по координатам вершин полигонов

            # Если полигон касается границ квадрата (пересекается квадратом), то он не аннотируется
            # Если пиксель лежит на правой стороне любого квадрата
            if ((n == 0 or n == 3) and x == square_size) or ((n == 1 or n == 4) and x == square_size * 2) or ((n == 2 or n == 5) and x == square_size * 3):
                # x_new = 1.0        # Координата x_new в относительных единицах = 1.0
                polygon_cut = True # Полигон пересекает квадрат
                break              # Если полигон пересекает квадрат, то этот полигон не аннотируется
            # Если пиксель лежит на нижней стороне любого квадрата
            if ((n == 0 or n == 1 or n == 2) and y == square_size) or ((n == 3 or n == 4 or n == 5) and y == square_size + y_step):
                # y_new = 1.0        # Координата y_new в относительных единицах = 1.0
                polygon_cut = True # Полигон пересекает квадрат
                break              # Если полигон пересекает квадрат, то этот полигон не аннотируется
            # Если пиксель лежит на левой стороне любого квадрата
            if ((n == 0 or n == 3) and x == 0) or ((n == 1 or n == 4) and x == square_size * 1) or ((n == 2 or n == 5) and x == square_size * 2):
                polygon_cut = True # Полигон пересекает квадрат
                break              # Если полигон пересекает квадрат, то этот полигон не аннотируется
            # Если пиксель лежит на верхней стороне любого квадрата
            if ((n == 0 or n == 1 or n == 2) and y == 0) or ((n == 3 or n == 4 or n == 5) and y == y_step):
                polygon_cut = True # Полигон пересекает квадрат
                break              # Если полигон пересекает квадрат, то этот полигон не аннотируется

            if polygon_cut == False:
                # Преобразование координат в пикселях в доли квадрата
                if n <= 2:              # Верхний ряд нарезанных кадров
                    # Преобразование координат в пикселях в доли квадрата
                    x_new = (x % square_size) / square_size
                    y_new = (y % square_size) / square_size
                else:                   # Нижний ряд нарезанных файлов
                    # Преобразование координат в пикселях в доли квадрата
                    x_new = (x % square_size) / square_size
                    y_new = ((y - y_step) % square_size) / square_size

                transformed_polygon.append(f"{x_new} {y_new}")         # Запись координат вершин полигонов
        if polygon_cut == False:  # Если полигон не пересекает квадрат
            transformed_polygons.append(' '.join(transformed_polygon)) # Запись строк (полигонов)

    return transformed_polygons # Функция возвращает список координат вершин полигонов в долях квадрата


# Функция нарезки полигонов пересекающим квадратом
# Если полигон пересекается с квадратом, находим их пересечение как пересечение множеств
def cut_polygons_with_square(polygons, n):
    # Функция принимает список полигонов, ширину и высоту кадра,
    # размер пересекающего квадрата и текущий номер квадрата n

    sliced_polygons = []    # Список нарезанных полигонов

    # Выбор левой верхней вершины квадрата
    if n == 0: # Если номер текущего квадрата = 0
        x = 0
        y = 0
    if n == 1:
        x = square_size
        y = 0
    if n == 2:
        x = square_size * 2
        y = 0
    if n == 3:
        x = 0
        y = y_step
    if n == 4:
        x = square_size
        y = y_step
    if n == 5:  # Если номер текущего квадрата = 5
        x = square_size * 2
        y = y_step

    # Квадрат это тоже полигон
    square = Polygon([(x, y), (x + square_size, y), (x + square_size, y + square_size), (x, y + square_size)])

    for polygon in polygons:     ######################
        if not polygon.is_valid:  # проверка на самопересечения   ####################
            polygon = polygon.buffer(0)  # исправление самопересечений ###############
        if polygon.geom_type == 'Point': ############
            print(new_file_prefix)

    if polygon.intersects(square):                  # Если внутри квадрата есть полигоны
        intersection = polygon.intersection(square) # Пересечение множеств (полигонов и квадрата)
        if intersection.geom_type == 'Polygon':     # Проверка находится ли внутри квадрата только один полигон
            intersection = [intersection]           # то полигон сохраняется в виде списка с одним полигоном
        elif intersection.geom_type == 'MultiPolygon': # Если внутри квадрата более одного полигона
            intersection = list(intersection.geoms)    # то полигоны сохраняются в виде списка полигонов
        elif intersection.geom_type == 'Point':
            print(new_file_prefix)
        elif intersection.geom_type == 'GeometryCollection':
            print(new_file_prefix)
        elif intersection.geom_type == 'LineString':
            print(new_file_prefix)

 #       for inter in intersection:                     # Проход по каждому элементу списка intersection
 #           if inter.geom_type != 'Point':   #########
 #           #    print(new_file_prefix)
 #               coords = inter.exterior.coords[:-1]        # Каждый элемент преобразуется в список координат вершин полигона, пересеченного квадратом
 #               sliced_polygons.append(coords)             # Полученный полигон добавляется в список полигонов пересеченных текущим квадратом
        for inter in intersection:
            if inter.geom_type == 'LineString':
                print(new_file_prefix)
            if inter.geom_type == 'GeometryCollection':
                print(new_file_prefix)
            if inter.geom_type != 'Point' and inter.geom_type != 'GeometryCollection':
                coords = inter.exterior.coords[:-1]
                sliced_polygons.append(coords)

    return sliced_polygons # Фунция возвращает полигоны, нарезанные квадратом


# Создать новую папку, если она не существует и удалить, если существует
if os.path.exists(new_labels_folder): # Если папка существует
    # Получаем список файлов в папке
    file_list = os.listdir(new_labels_folder)
    # Проходим по каждому файлу и удаляем его
    for file_name in file_list:
        file_path = os.path.join(new_labels_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)           # Удалить файл
    os.rmdir(new_labels_folder)            # удалить папку
if not os.path.exists(new_labels_folder):  # Если папка не существует
    os.makedirs(new_labels_folder)         # создать папку

# Обработка каждого файла аннотаций
for filename in os.listdir(labels_folder):  # Проход по всем файлам папки
    if filename.endswith(".txt"):           # Проверка окончания имени файла
        file_path = os.path.join(labels_folder, filename) # Путь к файлу
        new_file_prefix = os.path.splitext(filename)[0]   # Скопировать и сохранить имя исходного файла

        with open(file_path, "r") as file:  # Открыть файл для чтения
            annotations = file.readlines()  # Читать строки (аннотации)

    # Функци нарезки полигонов пересекающим квадратом
    for n in range(0, 6):  # Проходим по всем квадратам кадра
        for i, annotation in enumerate(annotations):  # Проход по всем строкам (полигонам) файла аннотаций
            annotation = annotation.strip()           # Убрать начальные и оконечные пробелы в строке
            # Прочитать имя класса и координаты
            class_id, *coordinates = map(float, annotation.split())
            # Координаты x y полигонов
            polygon_coords = [(coordinates[j], coordinates[j + 1]) for j in range(0, len(coordinates), 2)]

            # Функция преобразования координат из долей кадра в пиксели
            polygon = Polygon(convert_relative_to_pixels(polygon_coords))

            # Функция нарезки полигонов пересекающим квадратом
            sliced_polygons = cut_polygons_with_square([polygon], n)

            # Преобразование координат нарезанных полигонов из пикселей в доли пересекающего квадрата
            converted_polygons = convert_coordinates_to_ratios(sliced_polygons, n)

            # Запись полигонов в файлы с расширением "_n.txt", n - номер разрезанного файла аннотации в исходном кадре
            for converted_polygon in converted_polygons: # Проходим по всем полигонам в нарезанном квадрате
                new_file_path = os.path.join(new_labels_folder, f"{new_file_prefix}_{n}.txt") # Формирование пути к записываемым файлам
                with open(new_file_path, "a") as new_file:                      # Открыть файл для записи
                    new_file.write(f"{int(class_id)} {converted_polygon}\n")    # Запись файла


unique_frame_525


TypeError: ignored